In [21]:
import pandas as pd

def clean_column_name(df:pd.DataFrame, cols: list[str]| None = None) -> pd.DataFrame:
    target_cols = df.columns if cols is None else cols
    new_cols = []
    for col in df.columns:
        if col in target_cols:
            new_col = col.strip()
            new_col = new_col.replace(' ', '_') 
            new_col = new_col.upper() 
            new_cols.append(new_col)
        else:
            new_cols.append(col)
    df.columns = new_cols
    
    return df


In [22]:
df = pd.DataFrame({
    " First Name ": ["Alice", "Bob"],
    "Last Name": ["Smith", "Lee"],
    "age": [25, 30]
})

print("原列名:", df.columns.tolist())

df1 = clean_column_name(df.copy())
print("所有列:", df1.columns.tolist())

df2 = clean_column_name(df.copy(), cols=[" First Name ", "age"])
print("部分列:", df2.columns.tolist())


原列名: [' First Name ', 'Last Name', 'age']
所有列: ['FIRST_NAME', 'LAST_NAME', 'AGE']
部分列: ['FIRST_NAME', 'Last Name', 'AGE']


In [23]:
def formatting(df: pd.DataFrame, mapping: list[dict]) -> pd.DataFrame:
    for m in mapping:
        cols = m.get('Column', [])
        trans_type = m.get('trans_type', None)
        
        if trans_type == 'str':
            for col in cols:
                if col in df.columns:
                    df[col] = df[col].astype(str)     
                    df[col] = df[col].str.upper()     
                    df[col] = df[col].str.strip()      
                    df[col] = df[col].str.replace(" ", "_")
        
        elif trans_type == 'int':
            for col in cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64") 
        
        elif trans_type == 'float':
            for col in cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors="coerce").round(4)
        
        elif trans_type == 'bool':
            for col in cols:
                if col in df.columns:
                    df[col] = df[col].astype("boolean")
            
        elif trans_type == 'percent':
            for col in cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors="coerce") * 100
                    df[col] = df[col].round(2)
                    df[col] = df[col].apply(lambda x: f"{x}%" if pd.notna(x) else pd.NA)
        
        elif trans_type == 'date':
            date_format = m.get("format", "YYYY-MM-DD")  
            
            from dateutil import parser
            def parse_date(val):
                try:
                    return parser.parse(str(val), dayfirst=False, yearfirst=False)
                except Exception:
                    return pd.NaT
                
            for col in cols:
                if col in df.columns:
                    df[col] = df[col].apply(parse_date)
                    
                    if date_format == "YYYY-MM-DD":
                        df[col] = df[col].dt.strftime("%Y-%m-%d")
                    elif date_format == "DD_MM_YY":
                        df[col] = df[col].dt.strftime("%d-%m-%y")
                    elif date_format == "MM-YY":
                        df[col] = df[col].dt.strftime("%m-%y")
                        
        elif trans_type == "scale":
            factor = m.get("factor", 1)
            operation = m.get("operation", "mul")
            for col in cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors="coerce")  # 转数值
                    if operation == "mul":
                        df[col] = df[col] * factor
                    elif operation == "div":
                        df[col] = df[col] / factor
                    elif operation == "add":
                        df[col] = df[col] + factor
                    elif operation == "sub":
                        df[col] = df[col] - factor

                        
        elif trans_type == 'missing':
            strategy = m.get("strategy", None)
            for col in cols:
                if col in df.columns:
                    if strategy == "mean":
                        df[col] = df[col].fillna(df[col].mean())
                    elif strategy == "median":
                        df[col] = df[col].fillna(df[col].median())
                    elif strategy == "nan":
                        df[col] = df[col].fillna(pd.NA)
        
        elif trans_type == "outlier":
            method = m.get("method", "zscore")
            replace = m.get("replace", "nan")
            threshold = m.get("threshold", 3)
            for col in cols:
                if col in df.columns:
                    series = df[col]
                    if method == "zscore":
                        mean, std = series.mean(), series.std()
                        mask = abs(series - mean) > threshold * std
                    elif method == "iqr":
                        q1, q3 = series.quantile([0.25, 0.75])
                        iqr = q3 - q1
                        mask = (series < q1 - threshold * iqr) | (series > q3 + threshold * iqr)
                   
                    if replace == "mean":
                        df.loc[mask, col] = mean
                    elif replace == "median":
                        df.loc[mask, col] = series.median()
                    elif replace == "nan":
                        df.loc[mask, col] = pd.NA
                        
    df = df.astype(object).where(df.notna(), float("nan"))                    
    return df                    


In [18]:
df = pd.DataFrame({
    "name": [" Alice ", "bob", None, "Charlie Brown"],     # str
    "score": [3.1415926, 2.7, None, 999],                  # float + outlier
    "rank": [1.0, None, 2.0, 100],                         # int + missing
    "growth": [0.1234, 0.98765, None, 0.5],                # percent + missing
    "birthday": ["2024/01/05", "05-09-2025", None, "2025-12-31"],  # date
    "flag": [1, 0, None, 1]                                # bool
})

print("原始数据：")
print(df)

# 定义 mapping
mapping = [
    {"Column": ["name"], "trans_type": "str"},
    {"Column": ["score"], "trans_type": "float"},
    {"Column": ["rank"], "trans_type": "int"},
    {"Column": ["flag"], "trans_type": "bool"},
    {"Column": ["growth"], "trans_type": "percent"},
    {"Column": ["birthday"], "trans_type": "date", "format": "DD_MM_YY"},
    {"Column": ["rank"], "trans_type": "missing", "strategy": "median"},
    {"Column": ["score"], "trans_type": "outlier", "method": "zscore", "threshold": 3, "replace": "median"}
]

# 调用函数
df_fmt = formatting(df.copy(), mapping)

print("\n格式化后的数据：")
print(df_fmt)

print("\n数据类型：")
print(df_fmt.dtypes)


原始数据：
            name       score   rank   growth    birthday  flag
0         Alice     3.141593    1.0  0.12340  2024/01/05   1.0
1            bob    2.700000    NaN  0.98765  05-09-2025   0.0
2           None         NaN    2.0      NaN        None   NaN
3  Charlie Brown  999.000000  100.0  0.50000  2025-12-31   1.0

格式化后的数据：
            name   score rank  growth  birthday   flag
0          ALICE  3.1416    1  12.34%  05-01-24   True
1            BOB     2.7    2  98.76%  09-05-25  False
2           NONE     NaN    2     NaN       NaN    NaN
3  CHARLIE_BROWN   999.0  100   50.0%  31-12-25   True

数据类型：
name        object
score       object
rank        object
growth      object
birthday    object
flag        object
dtype: object


In [20]:
df1 = pd.DataFrame({
    "id": [1, 2, 3],
    "amount": [10, 20, 30]   
})
mapping = [
    {"Column": ["amount"], "trans_type": "scale", "factor": 10000, "operation": "mul"}
]

df1_fmt = formatting(df1.copy(), mapping)
print(df1_fmt)


  id  amount
0  1  100000
1  2  200000
2  3  300000
